<a href="https://colab.research.google.com/github/felipemontano10/nlp/blob/master/hw2part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**HOMEWORK 2 - Part 1**
##J. Felipe Montano-Campos





# Process the Sim Lex999 Data

In [0]:
from random import seed
from random import randint


In [0]:
filename = "Sim_Lex999.txt"
file = open(filename, "r")
text = file.read()
file.close()
with open(filename) as file_in:
    lines = []
    for line in file_in:
        lines.append(line)

Let's get three random rows

In [0]:
value1 = randint(2, 1000)
value2 = randint(2, 1000)
value3 = randint(2, 1000)
print(value1)
print(value2)
print(value3)



975
197
679


In [0]:
print(lines[value1])
print(lines[value2])
print(lines[value3])

go:sell:0.97:v:v

winter:season:6.27:n:n

apple:lemon:4.05:n:n



##      Word Embedding and Similarity Measurement

**Method 1:** Let's run the code Universal Sentence Coder to get the Word Embedding using this method 



In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



In [0]:
message_embedding_snippet

'-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904'

In [0]:
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

In [0]:
message_embeddings

array([[-1.64045133e-02, -4.48384844e-02, -3.78069095e-03,
         4.68277894e-02, -7.71008851e-03, -5.05571701e-02,
        -6.78127445e-03, -3.27876322e-02, -6.28904551e-02,
        -4.78958301e-02,  5.76800331e-02, -5.30807972e-02,
         2.26386786e-02,  2.00913250e-02, -1.91726331e-02,
         1.45138241e-02, -9.16321948e-03, -3.39176925e-03,
        -2.80498969e-03, -3.79767418e-02,  4.18685153e-02,
        -6.24692850e-02, -2.87507810e-02, -3.40368375e-02,
        -3.47199216e-02, -2.53357105e-02, -4.59449776e-02,
        -7.09898174e-02, -6.53066039e-02,  3.80705930e-02,
        -3.92757617e-02, -2.31163409e-02, -3.94491926e-02,
         1.65286213e-02,  3.73129509e-02, -5.67618310e-02,
        -3.04891113e-02,  5.97833432e-02, -2.56873239e-02,
        -6.43153442e-04,  5.93997464e-02,  5.53347953e-02,
         2.13192087e-02, -5.59708215e-02,  4.60403711e-02,
         2.85874866e-02,  3.25253457e-02, -7.22681209e-02,
         5.07617891e-02, -1.03512416e-02, -5.29682003e-0

**Method 2:** Let's run the code for Filling Blank Using Context (CBOW) to get the Word Embedding using this method  


Download the data

In [0]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-19 16:32:37--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  --.-KB/s    in 0.1s    

2020-01-19 16:32:38 (218 MB/s) - ‘reviews.txt’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the who

In [0]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-19 16:32:42--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.03s   

2020-01-19 16:32:43 (7.52 MB/s) - ‘labels.txt’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [0]:
vocab[11]

'roofie'

In [0]:
vocab

['debuting',
 'simian',
 'wedlock',
 'frocked',
 'pierce',
 'larson',
 'looneys',
 'headstrong',
 'constable',
 'picturehe',
 'completer',
 'roofie',
 'adt',
 'marketplaces',
 'pronouncement',
 'poolguy',
 'scarman',
 'ooky',
 'pinchers',
 'emperor',
 'glamour',
 'hairs',
 'vendors',
 'mmt',
 'notebooks',
 'fixate',
 'hamburglar',
 'recorders',
 'blanchett',
 'june',
 'kato',
 'bedding',
 'serato',
 'melodramas',
 'crreeepy',
 'danny',
 'zacatecas',
 'nosebleeds',
 'neri',
 'boooooo',
 'reay',
 'pruning',
 'pyschosis',
 'masonic',
 'torrid',
 'longo',
 'thorwald',
 'grape',
 'ambiance',
 'trill',
 'yumiko',
 'concept',
 'barris',
 'dynamically',
 'scanning',
 'vaguely',
 'snooker',
 'fibreglass',
 'mayron',
 'ryaburi',
 'havel',
 'boohooo',
 'sabriye',
 'intellectual',
 'eion',
 'turkic',
 'geysers',
 'khemu',
 'ignoble',
 'upper',
 'soba',
 'breads',
 'renting',
 'richmond',
 'beltway',
 'tually',
 'externally',
 'enlarges',
 'slogging',
 'mstie',
 'gonorrhea',
 'immigrating',
 'dicke

Initialize Weights

In [0]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]


Run the Model

In [0]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.8319583333333334%
Iter:1 Progress:95.99% Training Accuracy:0.8659791666666666%
Test Accuracy:0.852


Let's get the vector for a word

In [0]:
target = "dentist"
target_index = word2index[target]
weights_0_1[target_index]

array([-0.09973372, -0.10232338,  0.01798506,  0.02537488, -0.08167032,
        0.01467445,  0.07548596,  0.03126983, -0.03892901, -0.08077518,
        0.03842601, -0.12423577, -0.04707818, -0.00554906,  0.05786078,
       -0.0130101 ,  0.00494964, -0.03876167, -0.08529449,  0.05044516,
        0.01437539, -0.01743565,  0.00906359, -0.04553606,  0.09406705,
       -0.0282953 , -0.07438836, -0.00335827, -0.02444413,  0.06187729,
        0.08737949,  0.04218282, -0.10402784,  0.039824  ,  0.05019901,
       -0.06451833,  0.0657622 ,  0.03233859, -0.02599397,  0.04196617,
        0.08920156, -0.09821822, -0.01899641, -0.03927815, -0.10494433,
        0.06849393, -0.04824188, -0.01827022,  0.01878811,  0.00843129,
       -0.01776097,  0.05270098, -0.04176583,  0.02303049,  0.07280976,
       -0.08688769,  0.04880816, -0.04933059, -0.03159854,  0.07232905,
       -0.04558009,  0.12114611,  0.04254438, -0.01163467, -0.02094731,
        0.07508868, -0.05048899,  0.06905061,  0.01154609, -0.11

**Method 3:** Let's run the code Simplest word2vec to get the Word Embedding using this method  


In [0]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-19 16:35:11--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   872KB/s    in 36s     

2020-01-19 16:35:47 (855 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [0]:
!unzip text8.zip

Archive:  text8.zip
  inflating: text8                   


In [0]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [0]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8', sg=0, size=200, window=8, negative=25, hs=0, sample=1e-4, workers=20, min_count=5, iter=15)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's get the vector for a word

In [0]:
vector = model.wv['computer']
print(vector)

[ 0.48510426 -0.7160051  -0.6854017   0.955343    0.4747217  -0.52960235
 -0.26270095  0.85960984  0.26024544 -0.11315063  0.41707852  0.16497087
  0.13923208 -1.0012696   0.25271562 -0.63061744  0.49414426  0.01971136
 -0.40476573  0.07290158  1.0257103  -0.30192968  1.0847229  -1.0847398
  0.58647346  0.89671993 -0.18600236  0.9819717   1.4296737  -0.28193998
 -0.4078654  -0.01951197 -0.61289614 -0.70960593 -0.56282073 -0.20745102
 -0.2629219   0.9169005   0.15024388 -0.45934585  1.1095932  -0.6293792
  1.2036548   0.11496879  0.12684397 -0.15024748 -0.5992606   0.15775152
 -1.602282   -0.57597136  0.06771424 -0.46035585 -0.4733512   0.89628077
 -0.88359505  0.45431048  0.44235113 -0.47377422  0.28555453 -0.49019748
 -0.9284869   0.36679336 -1.0388595   0.4867893  -0.4811202  -0.48730654
  0.7876218  -0.9157782   0.00262798  0.20349774  0.68329096 -0.2987786
 -0.23044682  0.5578539  -0.9060894  -0.52301073 -0.78355193 -0.48432544
  0.4053386  -0.10769942  0.10152813  0.02598855  0.59

# Cosine Similarity


In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

**Using the embedding word in Method 1**

Pair of words number 1 

In [0]:
print(lines[value1])

go:sell:0.97:v:v



In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "go"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: go
Embedding size: 512
Embedding: [-0.019513163715600967, -0.00862062256783247, 0.02588113397359848, ...]



In [0]:
vector1 = message_embedding
vector1 = np.asarray(vector1)
print(vector1.shape)
vector1 = vector1.reshape(1,512)
print(vector1.shape)

(512,)
(1, 512)


In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "sell"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: sell
Embedding size: 512
Embedding: [0.0733080804347992, -0.02458847314119339, -0.04516470059752464, ...]



In [0]:
vector2 = message_embedding
vector2 = np.asarray(vector2)
print(vector2.shape)
vector2 = vector2.reshape(1,512)
print(vector2.shape)

(512,)
(1, 512)


In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.38461383]]


Pair of words number 2

In [0]:
print(lines[value2])

winter:season:6.27:n:n



In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "winter"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: winter
Embedding size: 512
Embedding: [-0.06278520077466965, 0.044713642448186874, 0.014479436911642551, ...]



In [0]:
vector1 = message_embedding
vector1 = np.asarray(vector1)
print(vector1.shape)
vector1 = vector1.reshape(1,512)
print(vector1.shape)

(512,)
(1, 512)


In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "season"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: season
Embedding size: 512
Embedding: [-0.07500195503234863, 0.003996224608272314, -0.05541529878973961, ...]



In [0]:
vector2 = message_embedding
vector2 = np.asarray(vector2)
print(vector2.shape)
vector2 = vector2.reshape(1,512)
print(vector2.shape)

(512,)
(1, 512)


In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.70394513]]


Pair of words number 3 

In [0]:
print(lines[value3])

apple:lemon:4.05:n:n



In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "apple"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: apple
Embedding size: 512
Embedding: [-0.03251326084136963, 0.04001666232943535, -0.024280602112412453, ...]



In [0]:
vector1 = message_embedding
vector1 = np.asarray(vector1)
print(vector1.shape)
vector1 = vector1.reshape(1,512)
print(vector1.shape)

(512,)
(1, 512)


In [0]:
# Compute a representation for each message, showing various lengths supported.
word = "lemon"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: lemon
Embedding size: 512
Embedding: [-0.06213806942105293, -0.027360964566469193, -0.009974876418709755, ...]



In [0]:
vector2 = message_embedding
vector2 = np.asarray(vector2)
print(vector2.shape)
vector2 = vector2.reshape(1,512)
print(vector2.shape)

(512,)
(1, 512)


In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.53498339]]


**Using the embedding word in Method 2**

Pair of words number 1

In [0]:
print(lines[value1])

go:sell:0.97:v:v



In [0]:
target1 = "go"
target2 = "sell"
target_index1 = word2index[target1]
target_index2 = word2index[target2]
vector1 = weights_0_1[target_index1]
vector1 = vector1.reshape(1,100)
vector2 = weights_0_1[target_index2]
vector2 = vector2.reshape(1,100)

In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.18424729]]


Pair of words number 2

In [0]:
print(lines[value2])

winter:season:6.27:n:n



In [0]:
target1 = "winter"
target2 = "season"
target_index1 = word2index[target1]
target_index2 = word2index[target2]
vector1 = weights_0_1[target_index1]
vector1 = vector1.reshape(1,100)
vector2 = weights_0_1[target_index2]
vector2 = vector2.reshape(1,100)

In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.30536699]]


Pair of words number 3

In [0]:
print(lines[value3])

apple:lemon:4.05:n:n



In [0]:
target1 = "apple"
target2 = "lemon"
target_index1 = word2index[target1]
target_index2 = word2index[target2]
vector1 = weights_0_1[target_index1]
vector1 = vector1.reshape(1,100)
vector2 = weights_0_1[target_index2]
vector2 = vector2.reshape(1,100)

In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[-0.09574697]]


**Using the embedding word in Method 3**

Pair of words number 1 

In [0]:
print(lines[value1])

go:sell:0.97:v:v



In [0]:
vector1 = model.wv["go"]
vector1 = vector1.reshape(1,200)
vector2 = model.wv["sell"]
vector2 = vector2.reshape(1,200)



In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.99048984]]


Pair of words number 2

In [0]:
print(lines[value2])

winter:season:6.27:n:n



In [0]:
vector1 = model.wv["winter"]
vector1 = vector1.reshape(1,200)
vector2 = model.wv["season"]
vector2 = vector2.reshape(1,200)

In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.99857444]]


Pair of words number 3

In [0]:
print(lines[value3])

apple:lemon:4.05:n:n



In [0]:
vector1 = model.wv["apple"]
vector1 = vector1.reshape(1,200)
vector2 = model.wv["lemon"]
vector2 = vector2.reshape(1,200)

In [0]:
cos_lib = cosine_similarity(vector1, vector2)
print(cos_lib)

[[0.9743163]]
